<a href="https://colab.research.google.com/github/anomaitech/GeoApp/blob/main/C5_Cell_Phone_Tower_Placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>